<a href="https://colab.research.google.com/github/stefandanielachirei/licenta_2023_Ceica_Sergiu/blob/main/WiDS_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# read dataset
dataSet = pd.read_csv("/content/train.csv")

# check data for NaN
print(dataSet.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6090 entries, 0 to 6089
Data columns (total 83 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   patient_id                             6090 non-null   int64  
 1   patient_race                           2764 non-null   object 
 2   payer_type                             5492 non-null   object 
 3   patient_state                          6075 non-null   object 
 4   patient_zip3                           6090 non-null   int64  
 5   patient_age                            6090 non-null   int64  
 6   patient_gender                         6090 non-null   object 
 7   bmi                                    1919 non-null   float64
 8   breast_cancer_diagnosis_code           6090 non-null   object 
 9   breast_cancer_diagnosis_desc           6090 non-null   object 
 10  breast_cancer_diagnosis_year           6089 non-null   float64
 11  meta

In [3]:
# check for the number of missing values
missingValues = dataSet.isnull().sum()
pd.set_option('display.max_rows', 500)
display(missingValues)

patient_id                                  0
patient_race                             3326
payer_type                                598
patient_state                              15
patient_zip3                                0
patient_age                                 0
patient_gender                              0
bmi                                      4171
breast_cancer_diagnosis_code                0
breast_cancer_diagnosis_desc                0
breast_cancer_diagnosis_year                1
metastatic_cancer_diagnosis_code            1
metastatic_first_treatment                  1
metastatic_first_treatment_type          2372
metastatic_first_novel_treatment         6048
metastatic_first_novel_treatment_type    6048
region                                     44
division                                   44
population                                  1
density                                     1
age_median                                  1
age_under_10                      

In [4]:
# Get general description of data
dataSet.describe()

,patient_id,patient_zip3,patient_age,bmi,breast_cancer_diagnosis_year,population,density,age_median,age_under_10,age_10_to_19,...,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,treatment_pd
count,6090.000000,6090.000000,6090.000000,1919.000000,6089.000000,6089.000000,6089.000000,6089.000000,6089.000000,6089.000000,...,6089.000000,6089.000000,6089.000000,6089.000000,6088.000000,6088.000000,6089.000000,6089.000000,6089.000000,6089.000000
mean,545331.323645,522.059113,54.331363,29.461918,2016.494334,19809.664148,1672.527089,40.724393,11.090219,12.855102,...,5.306488,6.469138,17.804551,13.479666,13.362461,4.600771,28.110536,8.606259,7.117613,132.610938
std,259819.076054,291.579396,10.945147,5.674036,1.045456,13447.456585,3702.687589,4.042574,1.520224,1.887636,...,5.883371,3.576789,17.589832,3.834710,5.745059,6.496853,4.834573,4.320390,3.131095,182.656529
min,100051.000000,6.000000,22.000000,14.000000,2015.000000,830.000000,0.821739,20.600000,0.000000,6.314286,...,0.011628,0.706250,0.194444,4.600000,3.433333,0.000000,12.460784,1.165385,1.200000,0.000000
25%,321267.500000,294.000000,47.000000,25.000000,2016.000000,8917.000000,166.050000,37.734426,10.108571,11.722078,...,1.302857,3.777273,4.433333,10.327273,9.324638,0.878049,25.013889,5.497143,4.972222,35.000000
50%,542817.500000,488.000000,55.000000,29.000000,2017.000000,18037.000000,612.507937,40.768605,11.003509,12.897826,...,3.236508,5.578689,11.434483,13.093333,12.122857,2.602128,27.828125,7.441772,7.033333,70.000000
75%,768701.250000,782.000000,62.000000,33.550000,2017.000000,27537.000000,1612.772727,43.175862,12.143396,13.948718,...,7.704348,8.680882,24.485185,15.611765,16.672727,5.958824,31.045238,10.771429,8.777778,133.000000
max,999269.000000,999.000000,91.000000,43.750000,2018.000000,71374.000000,29851.685710,57.417647,16.481818,35.300000,...,33.188889,26.429870,99.104348,35.155556,48.931818,74.067391,48.020000,27.566102,21.426667,1438.000000


In [5]:
# Get dataset column names
dataSet.columns

Index(['patient_id', 'patient_race', 'payer_type', 'patient_state',
       'patient_zip3', 'patient_age', 'patient_gender', 'bmi',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'breast_cancer_diagnosis_year', 'metastatic_cancer_diagnosis_code',
       'metastatic_first_treatment', 'metastatic_first_treatment_type',
       'metastatic_first_novel_treatment',
       'metastatic_first_novel_treatment_type', 'region', 'division',
       'population', 'density', 'age_median', 'age_under_10', 'age_10_to_19',
       'age_20s', 'age_30s', 'age_40s', 'age_50s', 'age_60s', 'age_70s',
       'age_over_80', 'male', 'female', 'married', 'divorced', 'never_married',
       'widowed', 'family_size', 'family_dual_income',
       'income_household_median', 'income_household_under_5',
       'income_household_5_to_10', 'income_household_10_to_15',
       'income_household_15_to_20', 'income_household_20_to_25',
       'income_household_25_to_35', 'income_household_35_to_5

In [6]:
# Now that we know many details about our dataset, we can go ahead and make sure our data is clean and does not have missing values.
# Some technics for data processing:
    # 1. Remove rows with missing data -> it can reduce the amount of data dramatically and affect your prediction result
    # 2. Impute missing values -> it can be used to impute missing values with meaningful values. Common imputation methods: using mean, mode, median
        # or advanced methods such as regression imputation. In simple words regression imputation uses existing data to predict missing data.

# For now lets simply delete rows with missing data

new_dataset = dataSet.dropna(axis=0)
new_dataset.describe()

,patient_id,patient_zip3,patient_age,bmi,breast_cancer_diagnosis_year,population,density,age_median,age_under_10,age_10_to_19,...,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,treatment_pd
count,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
mean,455221.750000,459.500000,49.250000,27.590000,2016.000000,16319.000000,528.991351,44.176091,10.308208,12.974158,...,3.288476,5.883854,13.057689,13.393024,10.438713,2.941490,29.137169,7.613815,8.789431,214.750000
std,301804.068117,261.333121,7.135592,5.550964,1.154701,8600.948901,402.369067,4.217444,1.849183,1.220484,...,2.625937,2.444151,7.612901,3.518495,3.654697,1.989343,4.249977,3.955632,2.259048,380.835376
min,162136.000000,208.000000,41.000000,20.120000,2015.000000,3787.000000,7.278261,41.472857,7.691304,11.478261,...,0.952174,3.182609,4.754286,9.321622,5.127027,1.369565,25.038095,4.365714,6.500000,23.000000
25%,269912.500000,303.250000,45.500000,25.280000,2015.000000,14744.500000,329.794565,42.014643,9.854969,12.287986,...,1.075901,4.144938,8.225528,11.696834,9.642626,1.609891,25.964959,4.391429,7.046711,23.750000
50%,399528.000000,408.000000,49.000000,28.620000,2016.000000,19111.500000,590.428571,42.383145,10.760238,13.074662,...,3.119286,6.077594,12.646568,13.225238,11.635342,2.327927,28.659576,6.817391,8.804950,25.000000
75%,584837.250000,564.250000,52.750000,30.930000,2017.000000,20686.000000,789.625357,44.544594,11.213477,13.760833,...,5.331861,7.816510,17.478728,14.921429,12.431429,3.659525,31.831786,10.039777,10.547671,216.000000
max,859695.000000,814.000000,58.000000,33.000000,2017.000000,23266.000000,927.830000,50.465217,12.021053,14.269048,...,5.963158,8.197619,22.183333,17.800000,13.357143,5.740541,34.191429,12.454762,11.047826,786.000000


In [7]:
# Select some features and save them in a list
# There are advanced technics for featutre selection such as Correlation Analysis,Recursive Feature Elimination (RFE), etc.
# For now we simply doing it manualy

important_features = ['patient_age', 'population', 'density']

# create a dataset based on selected features and new data set
X = new_dataset[important_features]
X.describe()

,patient_age,population,density
count,4.000000,4.000000,4.000000
mean,49.250000,16319.000000,528.991351
std,7.135592,8600.948901,402.369067
min,41.000000,3787.000000,7.278261
25%,45.500000,14744.500000,329.794565
50%,49.000000,19111.500000,590.428571
75%,52.750000,20686.000000,789.625357
max,58.000000,23266.000000,927.830000


In [20]:
X.head(5)

,patient_age,population,density
137,41,23266.0,743.557143
451,58,19826.0,437.300000
2561,51,3787.0,7.278261
4842,47,18397.0,927.830000


In [9]:
# Build simple model using sklearn
# create target
y = new_dataset.treatment_pd
dataset_model = DecisionTreeRegressor(random_state=1)
dataset_model.fit(X,y)

DecisionTreeRegressor(random_state=1)

In [10]:
#simple prediction example (its based on three features)

print(X)
print("The predictions are: ")
print(dataset_model.predict(X))

      patient_age  population     density
137            41     23266.0  743.557143
451            58     19826.0  437.300000
2561           51      3787.0    7.278261
4842           47     18397.0  927.830000
The predictions are: 
[786.  26.  23.  24.]


In [11]:
# lets compare our prediction with real values
print(f"New dataset treatment_pd column: \n{new_dataset.treatment_pd}\n\nPredictions: {dataset_model.predict(X)}")

New dataset treatment_pd column: 
137     786.0
451      26.0
2561     23.0
4842     24.0
Name: treatment_pd, dtype: float64

Predictions: [786.  26.  23.  24.]


In [12]:
# Evaluating the accuracy of the prediction using Mean Absolute Error (MAE)
# This is  "In Sample" score, meaning we used same sample for building and evaluating the model.

mean_absolute_error(y, dataset_model.predict(X))

0.0

In [14]:
# Read test data
test_data = pd.read_csv("/content/test.csv")

X_test = test_data[important_features]
X_test.describe()

,patient_age,population,density
count,2447.000000,2447.000000,2447.000000
mean,54.149571,19785.295055,1609.655851
std,11.462964,13518.227498,3537.533294
min,22.000000,866.000000,0.916667
25%,46.000000,8893.000000,161.782759
50%,54.000000,17681.000000,612.507937
75%,61.000000,27504.000000,1479.606250
max,91.000000,71374.000000,29851.685710


In [15]:
print(dataset_model.predict(X_test))


[ 24.  23.  26. ... 786.  24.  24.]


In [16]:
train_data_set = dataSet.iloc[:11845]

test_y = train_data_set.treatment_pd
mean_absolute_error(test_y, dataset_model.predict(X_test))

ValueError: ignored

In [17]:
from sklearn.ensemble import RandomForestRegressor

ran_forest_model = RandomForestRegressor(random_state=1)

train_y_ran = train_data_set.treatment_pd
X_train_ran = train_data_set[important_features]

ran_forest_model.fit(X_train_ran, train_y_ran)

ran_model_predict = ran_forest_model.predict(X_test)
ran_model_predict

ValueError: ignored

In [18]:
mean_absolute_error(test_y, ran_forest_model.predict(X_test))


AttributeError: ignored